In [29]:
#imports
import sys #you need this to add path to utils folder
sys.path.append('../utils') #now you can import from utils folder
# sys.path.append("../..")
from configs import config_general
from datasharders import datasharder_imu_joints
from datasetscopy import ImuJointPairDataset
from data_utils import fft_filter_signal, wavelet_filter_signal

In [28]:
help(config_general)

Help on class config_general in module configs:

class config_general(builtins.object)
 |  config_general(**kwargs)
 |
 |  Methods defined here:
 |
 |  __init__(self, **kwargs)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |
 |  __dict__
 |      dictionary for instance variables
 |
 |  __weakref__
 |      list of weak references to the object
 |
 |  ----------------------------------------------------------------------
 |  Data and other attributes defined here:
 |
 |  device = 'cpu'



In [30]:
#possible imports
import torch
import torch.nn as nn
import torchaudio.transforms as T
import numpy as np
from scipy.io import wavfile
from torch.utils.data import Dataset
import os
import matplotlib.pyplot as plt

In [31]:
#model class #MODEL
class DeepLSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(DeepLSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc1 = nn.Linear(hidden_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h_0 = torch.zeros(self.lstm.num_layers, x.size(0), self.lstm.hidden_size).to(x.device)
        c_0 = torch.zeros(self.lstm.num_layers, x.size(0), self.lstm.hidden_size).to(x.device)
        
        out, _ = self.lstm(x, (h_0, c_0))
        out = self.fc1(out)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [32]:
#config definition
#some config errors
config = config_general( #these are just example values we will change them according to model
    batch_size = 64 , #check
    epochs = 1000, #check
    lr = .002, #check
    scheduler = None , #check
    num_channels_imu = 3, #check
    num_channels_joints = 3, #check
    num_sessions = 1, #check
    seed = 42, #check
    data_folder_name = "../../datacollection/data", #check
    dataset_root = "../../datasets", #check
    #dataset_name = "example_dataset",
    dataset_train_name = "train_dataset", #check
    dataset_test_name = "test_dataset", #check
    num_patients = 3, #check
    window_length = 1000, #check
    #emg_transforms = [],
    imu_transforms = [fft_filter_signal], #check
    joint_transforms = [], #check
    hidden_size = 256,
    num_layers = 6, 
    input_size = 3,
    output_size = 3
)

In [ ]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(dataset_train, batch_size=64, shuffle=True)
test_dataloader = DataLoader(dataset_test, batch_size=64, shuffle=True)

In [ ]:
train_dataloader, test_dataloader

In [ ]:
train_features, train_labels = next(iter(train_dataloader))
print(f"Feature batch shape: {train_features.size()}")
print(f"Labels batch shape: {train_labels.size()}")

In [33]:
#main function that runs everything #TESTING
def main(config):
    
    #device definition?
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    model = DeepLSTMModel(input_size=config.input_size, hidden_size=config.hidden_size, num_layers=config.num_layers, output_size=config.output_size).to(device)
    criterion = nn.MSELoss(), #changes depending on model
    optimizer = torch.optim.Adam(model.parameters(), lr=config.lr), #changes depending on model
   
    
    #remake_dataset false if no changes made, true if changes need to be made 
    remake_dataset = True
    
    if(remake_dataset):
    
        #call datasharder on config 
        datasharder = datasharder_imu_joints(config, sample_rate=16000)
    
        #access data from datasharder
        training_data, testing_data = datasharder.load_data()
    
        #print shape
        print(training_data.data.shape)
        print(testing_data.data.shape)
    
        #save windowed data to config specified destination
        datasharder.save_windowed_data(training_data, "train")
        datasharder.save_windowed_data(testing_data, "test")
    
   #split dataset into train and test
    dataset_train = ImuJointPairDataset(config, "train")
    dataset_test = ImuJointPairDataset(config, "test")
    len(dataset_train), len(dataset_test)
    #STOP 
    
    imu_data_tensor,joint_data_tensor=dataset_train.__getitem__(1, "train")

    print(imu_data_tensor.shape, joint_data_tensor.shape)
    
    #convert to numpy
    imu_data = imu_data_tensor.numpy().squeeze(0)
    joint_data = joint_data_tensor.numpy().squeeze(0)
    import matplotlib.pyplot as plt
    import seaborn as sns
    sns.set(style="ticks")
    plt.plot(imu_data[:,1], label="imu")
    plt.plot(joint_data[:,1], label="joint")
    plt.legend()
    plt.show()


In [ ]:

    #train model
    train_losses, val_losses = train_model(model, train_loader, val_loader, num_epochs, criterion, optimizer, device)
    
    # Plot training and validation loss history
    plot_loss(train_losses, val_losses)
    
    # Evaluate model on validation and test sets and plot predictions
    val_inputs, val_targets, val_predictions = evaluate_model(model, val_loader, device)
    plot_predictions(val_inputs.squeeze(), val_targets.squeeze(), val_predictions.squeeze(), num_channels=3)

    test_inputs, test_targets, test_predictions = evaluate_model(model, test_loader, device)
    plot_predictions(test_inputs.squeeze(), test_targets.squeeze(), test_predictions.squeeze(), num_channels=3)
    
''''
def main(input_wav, target_wav, sample_rate=16000, hidden_size=256, num_layers=6, num_epochs=1000, batch_size=64, learning_rate=0.002):
   
    #device definition?
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    #dataset definition given exact imu and joint data files
    dataset = AudioDataset(input_wav, target_wav, sample_rate)
    
    #datasets defined and ready
    input_wav = dataset.input_wav
    target_wav = dataset.target_wav

    #plot standard data 
    plt.plot(input_wav)
    plt.title('Input IMU')
    plt.show()
    plt.plot(target_wav)
    plt.title('Target Joints')
    plt.show()
    
    # Split dataset, implement LOSOCV here
    train_size = int(0.8 * len(dataset))
    val_size = int(0.1 * len(dataset))
    test_size = len(dataset) - train_size - val_size
    train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

    #split data into loader objects
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    # Update input_size and output_size to match the number of features, define stuff like model, criterion and optimizer
    model = DeepLSTMModel(input_size=3, hidden_size=hidden_size, num_layers=num_layers, output_size=3).to(device)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    #train model
    train_losses, val_losses = train_model(model, train_loader, val_loader, num_epochs, criterion, optimizer, device)
    
    # Plot training and validation loss history
    plot_loss(train_losses, val_losses)
    
    # Evaluate model on validation and test sets and plot predictions
    val_inputs, val_targets, val_predictions = evaluate_model(model, val_loader, device)
    plot_predictions(val_inputs.squeeze(), val_targets.squeeze(), val_predictions.squeeze(), num_channels=3)

    test_inputs, test_targets, test_predictions = evaluate_model(model, test_loader, device)
    plot_predictions(test_inputs.squeeze(), test_targets.squeeze(), test_predictions.squeeze(), num_channels=3)
''''

In [34]:
#use main function accessing imu and joint data files directly
main(config)

AttributeError: 'config_general' object has no attribute 'num_patients'